In [1]:
import ee
ee.Initialize()
from geetools import batch

In [36]:
p1 = ee.Geometry.Point([-71,-42])
p2 = ee.Geometry.Point([-71,-43])
p3 = ee.Geometry.Point([-71,-44])

In [50]:
feat1 = ee.Feature(p1.buffer(5), {'site': 1})
feat2 = ee.Feature(p2.buffer(5), {'site': 2})
feat3 = ee.Feature(p3.buffer(5), {'site': 3})

In [51]:
fc = ee.FeatureCollection([feat1, feat2, feat3])

In [52]:
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']
#bands = ['B2', 'B3', 'B4']
# Use Landsat 8 surface reflectance data.

# Cloud masking function.
def maskL8sr(image):
    cloudShadowBitMask = ee.Number(2).pow(3).int()
    cloudsBitMask = ee.Number(2).pow(5).int()
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
    qa.bitwiseAnd(cloudsBitMask).eq(0))
    return image.updateMask(mask).select(bands).divide(10000)

# The image input data is a 2018 cloud-masked median composite.
reducer = ee.Reducer.median()

reducer_str = "median"

In [53]:
collection = ee.ImageCollection("LANDSAT/LE07/C01/T1_SR")

In [54]:
year = 2015
image = collection.filterDate(str(year) + '-01-01', str(year) + '-12-31').map(maskL8sr).reduce(reducer)

In [55]:
batch.image.toDriveByFeature(image, collection=fc, property='site', folder='tools_exportbyfeat', name='test', scale=10, dataType='float')

exporting test_site_1
exporting test_site_2
exporting test_site_3


[<Task EXPORT_IMAGE: test_site_1 (UNSUBMITTED)>,
 <Task EXPORT_IMAGE: test_site_2 (UNSUBMITTED)>,
 <Task EXPORT_IMAGE: test_site_3 (UNSUBMITTED)>]

In [21]:
ee.batch.Export.image.toDrive(image, 
    collection=fc, 
    property='site', 
    folder='tools_exportbyfeat1', 
    name='test', 
    scale=10, 
    dataType='float')



<Task EXPORT_IMAGE: myExportImageTask (UNSUBMITTED)>